In [1]:
# STEP 1: Install packages
!apt update && apt install -y tesseract-ocr tesseract-ocr-ben
!pip install pymupdf pytesseract pillow sentence-transformers faiss-cpu transformers

# STEP 2: Download Bengali language data manually
!wget -O ben.traineddata https://github.com/tesseract-ocr/tessdata/raw/main/ben.traineddata
!mkdir -p /content/tessdata
!mv ben.traineddata /content/tessdata/

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [1,853 kB]
Hit:5 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:6 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:7 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:9 https://r2u.stat.illinois.edu/ubuntu jammy/main amd64 Packages [2,770 kB]
Hit:10 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Get:11 http://security.ubuntu.com/ubuntu jammy-security/main amd64 Packages [3,160 kB]
Hit:12 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:13 http://archive.ubuntu.com/ubu

In [4]:
# ✅ Install dependencies (only once)
!pip install -q sentence-transformers faiss-cpu transformers

from sentence_transformers import SentenceTransformer
import faiss
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

# ✅ Load the full OCR text from the file
with open("Bangla_OCR_Full_Text.txt", "r", encoding="utf-8") as f:
    ocr_text = f.read()

# ✅ Chunk the text
chunk_size = 500
chunk_overlap = 100
chunks = [ocr_text[i:i + chunk_size] for i in range(0, len(ocr_text), chunk_size - chunk_overlap)]

# ✅ Create embeddings
embed_model = SentenceTransformer('sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2')
chunk_embeddings = embed_model.encode(chunks)

# ✅ Build vector index
dim = chunk_embeddings[0].shape[0]
index = faiss.IndexFlatL2(dim)
index.add(chunk_embeddings)

# ✅ Load answer generation model (can change to flan-t5-large for better answers)
qa_model_name = "google/flan-t5-base"
qa_tokenizer = AutoTokenizer.from_pretrained(qa_model_name)
qa_model = AutoModelForSeq2SeqLM.from_pretrained(qa_model_name)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [5]:
def ask_question(query, top_k=3):
    q_embed = embed_model.encode([query])
    D, I = index.search(q_embed, top_k)
    contexts = [chunks[i] for i in I[0]]
    full_context = "\n\n".join(contexts)

    prompt = (
        f"প্রশ্ন: {query}\n"
        f"নিচের প্রাসঙ্গিক তথ্যের ভিত্তিতে সংক্ষিপ্ত উত্তর দাও:\n\n"
        f"{full_context}\n\n"
        f"উত্তর:"
    )

    inputs = qa_tokenizer(prompt, return_tensors="pt", truncation=True, max_length=768)
    outputs = qa_model.generate(**inputs, max_new_tokens=150)
    answer = qa_tokenizer.decode(outputs[0], skip_special_tokens=True)

    return answer.strip() if answer.strip() else "দুঃখিত, প্রাসঙ্গিক তথ্য পাওয়া যায়নি।"

In [7]:
with open("Bangla_OCR_Full_Text.txt", "r", encoding="utf-8") as f:
    lines = f.readlines()

for line in lines:
    if "ভাগ্য দেবতা" in line or "অনুপম" in line:
        print(line.strip())

/ মানুষ আশা নিয়ে বেঁচে থাকে- অনুপমের দৃষ্টান্তে মানবজীবনের এই চিরন্তন সত্যদর্শন সম্পর্কে
১। অনুপমের বাবা কী করে জীবিকা নির্বাহ করতেন?
২। মামাকে ভাগ্য দেবতার প্রধান এজেন্ট বলার কারণ, তার-
৫. অনুপমের বয়স কত বছর?
রন গল্পের কথক চরিত্র অনুপমের আত্মসমালোচনা। পরিমাণ ও
ফলের মতো গুটি ওঠে। নিজের নিষ্ফল জীবনকে বোঝাতে অনুপমের ব্যবহৃত
ন্্ভর বালির মতন তিনি সাম্ফ্াণ অনুপম তার মামার চরিত্র-বৈশিষ্ট্য প্রসঙ্গে কথাটি বলেছে।
সংগীতসরস্বতীর পদ্মবন অনুপম নিজের বিবাহযাত্রার পরিস্থিতি বর্ণনায় সুরশূন্য
অনুপমের এই আত্মোপলক্ধি এখানে প্রকাশিত।
মামা বলিলেন, “অনুপম এখানে কি করিবে। ও সভায় গিয়ে বসুক।”
করিয়া বলিলেন, “অনুপম, যাও, তুমি সভায় গিয়া
পুরুষের জবানিতে লেখা গল্প। গল্পের কথক অনুপম বিশ শতকের দ্বিতীয় দশকের যুদ্ধসংলগ্ন সময়ের সেই
নিয়ে নারীর চরম অবমাননাকালে শত্তুনাথ সেনের কন্যা-সম্প্রদানে অসম্মতি গল্পটির শীর্ষ মুহূর্ত। অনুপম নিজের
অনুপমের আত্মবিবৃতির সূত্র ধরেই গল্পের নারী কল্যাণী অসামান্য হয়ে উঠেছে। গল্পটিতে পুরুষতন্ত্রের
১। অনুপমের বাবা কী করে জীবিকা নির্বাহ করতেন?
২। মামাকে ভাগ্য দেবতার প্রধান এজেন্ট বল

In [9]:
def get_top_k_context(query, top_k=3):
    q_embed = embed_model.encode([query])
    D, I = index.search(q_embed, top_k)

    # 🔍 Boost chunks that contain exact words (e.g., "অনুপম")
    keywords = ["অনুপম", "ভাগ্য দেবতা"]
    keyword_hits = [i for i, c in enumerate(chunks) if any(k in c for k in keywords)]

    # Boost keyword-based results (add to top if not already included)
    boosted_ids = list(dict.fromkeys(keyword_hits + list(I[0])))

    contexts = [chunks[i] for i in boosted_ids[:top_k]]
    return contexts, q_embed, [chunk_embeddings[i] for i in boosted_ids[:top_k]]

In [11]:
chunk_size = 200
chunk_overlap = 50
chunks = [ocr_text[i:i + chunk_size] for i in range(0, len(ocr_text), chunk_size - chunk_overlap)]

In [12]:
query = input("💬 প্রশ্ন লিখুন: ")
print("📄 উত্তর:", ask_question(query))

💬 প্রশ্ন লিখুন: কাকে অনুপমের ভাগ্য দেবতা বলে উল্লেখ করা হয়েছে?
📄 উত্তর: দুঃখিত, প্রাসঙ্গিক তথ্য পাওয়া যায়নি।


**Bonus task**

In [13]:
pip install fastapi uvicorn sentence-transformers faiss-cpu transformers

In [14]:
from fastapi import FastAPI, Request
from pydantic import BaseModel
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from sentence_transformers import SentenceTransformer
import faiss
import uvicorn

# ========== Setup ==========

app = FastAPI(title="Bangla RAG API")

# Load models
embed_model = SentenceTransformer('sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2')
qa_tokenizer = AutoTokenizer.from_pretrained("google/flan-t5-base")
qa_model = AutoModelForSeq2SeqLM.from_pretrained("google/flan-t5-base")

# Load your text chunks and build FAISS
with open("Bangla_OCR_Full_Text.txt", encoding="utf-8") as f:
    text = f.read()

chunks = [text[i:i+500] for i in range(0, len(text), 400)]
embeddings = embed_model.encode(chunks)
index = faiss.IndexFlatL2(embeddings[0].shape[0])
index.add(embeddings)

# ========== API Input Schema ==========
class Query(BaseModel):
    question: str

# ========== Main RAG Endpoint ==========
@app.post("/ask")
async def ask(query: Query):
    q_embed = embed_model.encode([query.question])
    D, I = index.search(q_embed, 3)
    context = "\n".join([chunks[i] for i in I[0]])

    prompt = (
        f"প্রশ্ন: {query.question}\n"
        f"প্রাসঙ্গিক তথ্য:\n{context}\n"
        f"উত্তর:"
    )

    inputs = qa_tokenizer(prompt, return_tensors="pt", truncation=True, max_length=768)
    outputs = qa_model.generate(**inputs, max_new_tokens=150)
    answer = qa_tokenizer.decode(outputs[0], skip_special_tokens=True)

    return {
        "question": query.question,
        "answer": answer.strip(),
        "context": context
    }

# ========== Run Locally ==========
# uvicorn main:app --reload

In [15]:
from sklearn.metrics.pairwise import cosine_similarity

def get_similarity_score(query: str, chunks: list):
    q_vec = embed_model.encode([query])
    c_vecs = embed_model.encode(chunks)
    sims = cosine_similarity(q_vec, c_vecs)
    return sims[0]

In [16]:
from fastapi import FastAPI
from pydantic import BaseModel
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from sentence_transformers import SentenceTransformer
import faiss
import uvicorn

# ========== Initialize FastAPI ==========
app = FastAPI(title="Bangla RAG API")

# ========== Load Models ==========
embed_model = SentenceTransformer('sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2')
qa_tokenizer = AutoTokenizer.from_pretrained("google/flan-t5-base")
qa_model = AutoModelForSeq2SeqLM.from_pretrained("google/flan-t5-base")

# ========== Load Text & Create Vector Index ==========
with open("Bangla_OCR_Full_Text.txt", encoding="utf-8") as f:
    full_text = f.read()

# Chunk the text
chunk_size = 500
chunk_overlap = 100
chunks = [full_text[i:i+chunk_size] for i in range(0, len(full_text), chunk_size - chunk_overlap)]

# Encode embeddings
embeddings = embed_model.encode(chunks)
dim = embeddings[0].shape[0]
index = faiss.IndexFlatL2(dim)
index.add(embeddings)

# ========== Request Schema ==========
class Query(BaseModel):
    question: str

# ========== Main API Endpoint ==========
@app.post("/ask")
async def ask_question(query: Query):
    # Embed the query
    q_embed = embed_model.encode([query.question])
    D, I = index.search(q_embed, 3)
    context_chunks = [chunks[i] for i in I[0]]
    context = "\n".join(context_chunks)

    # Prompt template
    prompt = (
        f"প্রশ্ন: {query.question}\n"
        f"প্রাসঙ্গিক তথ্য:\n{context}\n"
        f"উত্তর:"
    )

    # Generate answer
    inputs = qa_tokenizer(prompt, return_tensors="pt", truncation=True, max_length=768)
    outputs = qa_model.generate(**inputs, max_new_tokens=150)
    answer = qa_tokenizer.decode(outputs[0], skip_special_tokens=True).strip()

    # Groundedness check
    grounded = answer in context

    return {
        "question": query.question,
        "answer": answer,
        "grounded": grounded,
        "context": context
    }

# ========== Run with: ==========
# uvicorn main:app --reload


In [23]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

def get_top_k_context(query, top_k=3):
    q_embed = embed_model.encode([query])
    D, I = index.search(q_embed, top_k)
    contexts = [chunks[i] for i in I[0]]
    return contexts, q_embed, [chunk_embeddings[i] for i in I[0]]

def evaluate_groundedness(answer, contexts):
    answer_embed = embed_model.encode([answer])[0]
    context_embeds = embed_model.encode(contexts)
    similarities = cosine_similarity([answer_embed], context_embeds)[0]
    return round(float(np.mean(similarities)), 4)

def evaluate_relevance(query_embed, chunk_embeds):
    similarities = cosine_similarity(query_embed, chunk_embeds)[0]
    return round(float(np.mean(similarities)), 4)

def evaluate_rag(query, top_k=3):
    contexts, query_embed, chunk_embeds = get_top_k_context(query, top_k)
    answer = ask_question(query, top_k=top_k)
    groundedness = evaluate_groundedness(answer, contexts)
    relevance = evaluate_relevance(query_embed, chunk_embeds)

    print(f"\n📌 প্রশ্ন: {query}")
    print(f"📄 প্রাসঙ্গিক তথ্য (Top {top_k}):")
    for i, ctx in enumerate(contexts, 1):
        print(f"\n-- Chunk {i} --\n{ctx.strip()[:300]}...")  # only first 300 chars

    print(f"\n✅ জেনারেটেড উত্তর: {answer}")
    print(f"📊 গ্রাউন্ডেডনেস স্কোর: {groundedness}")
    print(f"📊 প্রাসঙ্গিকতার স্কোর: {relevance}")


In [24]:
query = "অনুপমের ভাষায় সুপুরুষ কাকে বলা হয়েছে?"
evaluate_rag(query)


📌 প্রশ্ন: অনুপমের ভাষায় সুপুরুষ কাকে বলা হয়েছে?
📄 প্রাসঙ্গিক তথ্য (Top 3):

-- Chunk 1 --
চিত্রশিল্পী, অনুসন্ধিৎসু বিশ্বপরিব্রাজক, দক্ষ সম্পাদক
এবং অসামান্য শিক্ষা-সংগঠক ও চিন্তক। নিজে প্রাতিষ্ঠানিক শিক্ষাগ্রহণে নিরুৎসাহী
হলেও 'বিশ্বভারতী' নামের বিশ্ববিদ্যালয়ের তিনি স্বাপ্নিক ও প্রতিষ্ঠাতা

 

 

 

কাব্যগ্রন্থ: মানসী, সোনার তরী, চিত্রা, চৈতালি, ক্ষণিকা, নৈবেদ্য, গীতাঞ্জলি, বলাকা,

পূর...

-- Chunk 2 --
আয়োজন
করিতে পারি নাই, ক্ষমা করিবেন। রাত হইয়া গেছে, আর আপনাদের কষ্ট বাড়াইতে ইচ্ছা করি না। এখন তবে-"

 

 

 

 

 

 

 

 

 


--- Page 11 ---
[11590 261
নস 1411475
অনলাইন ব্যাচ 13:0১

'বাংলা. ইংরেজি . আইসিটি

 

মামা বলিলেন, “তা, সভায় চলুন, আমরা তো প্রস্তুত আছি।”

শত্তুনাথ বলিলেন, “তবে আপনাদে...

-- Chunk 3 --
জানলার বাহিরে মুখে বাড়াইয়া প্রকৃতির শোভা দেখিতে লাগিলাম।

কানপুরে গাড়ি আসিয়া থামিল। মেয়েটি জিনিসপত্র বাঁধিয়া প্রস্তুত-স্টেশনে একটি হিন্দুস্থানি চাকর ছুটিয়া
আসিয়া ইহাদিগকে নামাইবার উদ্যোগ করিতে লাগিল।

মা তখন আর থাকিতে পারিলেন না। জিজ্ঞাসা করিলেন, “তোমার নাম কী ম